In [1]:
install.packages("jtools")
library("tidyverse") #load packages
library("jtools")
set.seed(1437) #Set the random number generator

Installing package into ‘/home/jupyter/R/x86_64-pc-linux-gnu-library/4.0’
(as ‘lib’ is unspecified)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.2     ✔ purrr   0.3.4
✔ tibble  3.0.3     ✔ dplyr   1.0.1
✔ tidyr   1.1.1     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.5.0

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



# Omitted Variables Bias

Omitted variables bias is a common issue in many models, and can be interpreted in several different ways.  Let's look at it in the simplest possible form, and then in the form related to selection.

Consider the following simple model, where $i$ is a population of Canadian workers, $y$ represents **wages**, $e$ represents **years of education**, and $t$ represents **job-tenure**.  

Now, let's suppose this is a classic Spence-style model, where individuals are paid their marginal product, and where education has no direct effect on productivity (and hence wages).  Instead, what really matters is $a$, **ability**.  In other words, the true model looks like:

$$y_i = \beta_0 + \beta_1 t + \beta_2 a + \epsilon_i$$

However, what we estimate is instead the following model:

$$y_i = B_0 + B_1 t + B_2 e + e_i$$

What we need to understand are the consequences of this decision - in particular what it implies about $B_1$ and $B_2$.  Some key questions are:

* Under what conditions will I correctly estimate $B_1 = \beta_1$?  Will I infer that $B_2 = 0$?  
* What are the consequences of leaving out a variable?  What happens to the rest of my model?

## Case 1: Simple Correlation

To begin, we will simulate a pair of models: one where $a$, $t$, and $e$ are correlated, and a version where $t$ is not related to ability or education

In [2]:
n = 500000 #sample size is 500,000

ability = runif(n, min = 0, max = 100) #ability is between 1 and 100 (normalized)

education = ability/5 + runif(n, min = 0, 2)
education = round(education, digits = 0)
# education is a strong function of ability

c1 = round(cor(education, ability), digits = 2)
cat("The correlation between education and ability is", c1, "\n")

# Tenure is also correlated with ability

tenure = ability/5 + runif(n, min = 0, 10)
tenure = round(tenure, digits = 0)

c2 = round(cor(tenure, ability), digits = 2)
cat("The correlation between tenure and ability is", c2, "\n")

# Now, let's create the true model

wage = 12 + 0.65*tenure + 0.10*ability + rnorm(n, mean = 0, sd = 2.5)

wage_data = tibble(wage, tenure, education, ability) #create data frame

## Now, let's make our uncorrelated data

tenure2 = runif(n, min = 0, 10) + 10 #same average tenure, but not related
c3 = round(cor(tenure2, ability), digits = 2)
cat("The correlation between tenure2 and ability is", c3, "\n")

wage2 = 12 + 0.65*tenure2 + 0.10*ability + rnorm(n, mean = 0, sd = 2.5)

wage_data2 = tibble(wage2, tenure2, education, ability) #create data frame


The correlation between education and ability is 0.99 
The correlation between tenure and ability is 0.89 
The correlation between tenure2 and ability is 0 


First, let's verify the true model; we should expect that $\beta_0 = 12$, $\beta_1 = 0.65$ and $\beta_2 = 0.10$.  Education, if included in the model, should have zero coefficient.

In [3]:
true_model <- lm(data = wage_data, formula = wage ~ tenure + ability)
summ(true_model)
true_model2 <- lm(data = wage_data, formula = wage ~ tenure + ability + education)
summ(true_model2)
true_model3 <- lm(data = wage_data2, formula = wage2 ~ tenure2 + ability + education)
summ(true_model3)

MODEL INFO:
Observations: 500000
Dependent Variable: wage
Type: OLS linear regression 

MODEL FIT:
F(2,499997) = 1897276.12, p = 0.00
R² = 0.88
Adj. R² = 0.88 

Standard errors: OLS
-------------------------------------------------
                     Est.   S.E.    t val.      p
----------------- ------- ------ --------- ------
(Intercept)         12.02   0.01   1283.79   0.00
tenure               0.65   0.00    530.32   0.00
ability              0.10   0.00    368.20   0.00
-------------------------------------------------

MODEL INFO:
Observations: 500000
Dependent Variable: wage
Type: OLS linear regression 

MODEL FIT:
F(3,499996) = 1264857.45, p = 0.00
R² = 0.88
Adj. R² = 0.88 

Standard errors: OLS
-------------------------------------------------
                     Est.   S.E.    t val.      p
----------------- ------- ------ --------- ------
(Intercept)         12.03   0.01   1110.41   0.00
tenure               0.65   0.00    530.32   0.00
ability              0.10   0.00     90.82   0.00
education           -0.01   0.01     -1.80   0.07
-------------------------------------------------

MODEL INFO:
Observations: 500000
Dependent Variable: wage2
Type: OLS linear regression 

MODEL FIT:
F(3,499996) = 314951.12, p = 0.00
R² = 0.65
Adj. R² = 0.65 

Standard errors: OLS
------------------------------------------------
                     Est.   S.E.   t val.      p
----------------- ------- ------ -------- ------
(Intercept)         12.01   0.02   587.07   0.00
tenure2              0.65   0.00   530.00   0.00
ability              0.10   0.00    89.81   0.00
education            0.00   0.01     0.82   0.41
------------------------------------------------

This is exactly what we expect, based on the true model

So, what happens if we leave out ability (since we cannot observe it)?

In [4]:
model1 <- lm(data = wage_data, formula = wage ~ tenure)
summ(model1)

MODEL INFO:
Observations: 500000
Dependent Variable: wage
Type: OLS linear regression 

MODEL FIT:
F(1,499998) = 2878501.71, p = 0.00
R² = 0.85
Adj. R² = 0.85 

Standard errors: OLS
-------------------------------------------------
                     Est.   S.E.    t val.      p
----------------- ------- ------ --------- ------
(Intercept)         11.02   0.01   1090.84   0.00
tenure               1.05   0.00   1696.61   0.00
-------------------------------------------------

This is very, very wrong - the coefficient on tenure is twice as large!

What's happening here?  When we omit a variable which is correlated, the "effect" spills over to other correlated variables.  You can see this in the other model, where tenure and ability were unrelated

In [5]:
model2 <- lm(data = wage_data2, formula = wage2 ~ tenure2)
summ(model2)

MODEL INFO:
Observations: 500000
Dependent Variable: wage2
Type: OLS linear regression 

MODEL FIT:
F(1,499998) = 120779.36, p = 0.00
R² = 0.19
Adj. R² = 0.19 

Standard errors: OLS
------------------------------------------------
                     Est.   S.E.   t val.      p
----------------- ------- ------ -------- ------
(Intercept)         17.01   0.03   594.97   0.00
tenure2              0.65   0.00   347.53   0.00
------------------------------------------------

The numbers are still correct!  This is because the omitted variable was unrelated to tenure variable.

## Adding Education into the Mix

Recall that education is a very strong function of ability; they are nearly perfectly correlated.  What does this do?

In [6]:
model1 <- lm(data = wage_data, formula = wage ~ tenure + education)
summ(model1)

MODEL INFO:
Observations: 500000
Dependent Variable: wage
Type: OLS linear regression 

MODEL FIT:
F(2,499997) = 1862443.79, p = 0.00
R² = 0.88
Adj. R² = 0.88 

Standard errors: OLS
-------------------------------------------------
                     Est.   S.E.    t val.      p
----------------- ------- ------ --------- ------
(Intercept)         11.49   0.01   1258.16   0.00
tenure               0.67   0.00    558.01   0.00
education            0.47   0.00    353.92   0.00
-------------------------------------------------

Look!  The cofficient on tenure is now nearly correct; the education one is completely wrong, but tenure is close.  Consider the uncorrelated model:

In [7]:
model2 <- lm(data = wage_data2, formula = wage2 ~ tenure2 + education)
summ(model2)

MODEL INFO:
Observations: 500000
Dependent Variable: wage2
Type: OLS linear regression 

MODEL FIT:
F(2,499997) = 460959.67, p = 0.00
R² = 0.65
Adj. R² = 0.65 

Standard errors: OLS
------------------------------------------------
                     Est.   S.E.   t val.      p
----------------- ------- ------ -------- ------
(Intercept)         11.58   0.02   577.50   0.00
tenure2              0.65   0.00   525.81   0.00
education            0.49   0.00   803.29   0.00
------------------------------------------------

Same result as before; education is wrong, but tenure is right.

### What is happening here?

When you omit a variable, the causal effect of that variable "spills over" to other variables in your model.  However, it spills over the MOST to variables which it is highly correlated; these "absorb" the effect.

In our example here, the ability variable spills over to both tenure and education - but because education is more correlated than tenure, the education variable is very wrong, while tenure is nearly right.

If you change the cell above to make education a perfect predictor of ability, you will see an interesting result; can you guess what it is?

## Omitted Variables and Selection

One way of thinking about this relationship is in terms of selection.

The key feature was the correlation, created in our model with:

```tenure = ability/5 + runif(n, min = 0, 10)```

You can think of this as an equation which represents selection:

$$t_i = \theta_0 + \theta_1 a_i + \eta_i$$

In our first model, $\theta_0  = 5$ and $\theta_1 = 0.2$ (and the residual, $\eta_i$ is uniform on $-5,5$, but that isn't important).  You can see this since we can actually estimate this model:

In [8]:
selection_model <- lm(data = wage_data, formula = tenure ~ ability)
summ(selection_model)

MODEL INFO:
Observations: 500000
Dependent Variable: tenure
Type: OLS linear regression 

MODEL FIT:
F(1,499998) = 1977773.15, p = 0.00
R² = 0.80
Adj. R² = 0.80 

Standard errors: OLS
------------------------------------------------
                    Est.   S.E.    t val.      p
----------------- ------ ------ --------- ------
(Intercept)         5.01   0.01    610.16   0.00
ability             0.20   0.00   1406.33   0.00
------------------------------------------------

In the other model, $\theta_1 = 0$.

In [9]:
selection_model2 <- lm(data = wage_data2, formula = tenure2 ~ ability)
summ(selection_model2)

MODEL INFO:
Observations: 500000
Dependent Variable: tenure2
Type: OLS linear regression 

MODEL FIT:
F(1,499998) = 0.03, p = 0.86
R² = 0.00
Adj. R² = -0.00 

Standard errors: OLS
-------------------------------------------------
                     Est.   S.E.    t val.      p
----------------- ------- ------ --------- ------
(Intercept)         15.01   0.01   1837.70   0.00
ability              0.00   0.00      0.17   0.86
-------------------------------------------------

This is exactly what we expected!  Now normally this kind of check is impossible, since ability is not observed - but this is what is occuring behind the scenes.  The cofficient on ability in this kind of model, determines the amount of spillover!